In [53]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
import asyncio
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
 






In [54]:
load_dotenv(override=True)

True

In [55]:
instructions_1  = """Your are a sales agent working for BetaLaunch.\n
It is a company which is oriented at providing software solutions for businesses such as web applications and mobile applications\n
Your write in a professional and serious tone\n
"""

instructions_2 = """Your are a sales agent working for BetaLaunch.\n
It is a company which is oriented at providing software solutions for businesses such as web applications and mobile applications\n
Your write in a engaging and responsive tone\n
"""

instructions_3 = """Your are a sales agent working for BetaLaunch.\n
It is a company which is oriented at providing software solutions for businesses such as web applications and mobile applications\n
Your write in a concise and detailed oriented tone. \n
"""


In [56]:
sales_agent_1 = Agent(name="Professional_sales_agent", instructions=instructions_1,)
sales_agent_2 = Agent(name="Engaging_sales_agent", instructions=instructions_2)
sales_agent_3 = Agent(name="details_oriented_sales_agent", instructions=instructions_3)

In [57]:
message = "Write a cold email"

with trace("Execution of multiple agents"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, message),
        Runner.run(sales_agent_2, message),
        Runner.run(sales_agent_3, message),
    )
    output = [result.final_output for result in results];


In [58]:
evaluation_agent_instructions = f"""
 You act as an evaluation agent, who's job is to evaluate the array of provided emails and pick the best one that a user might reply.\n
 Do not give any explanations, just reply with the selected email\n
"""

In [59]:
evaluation_agent = Agent(name="evaluation agnet", instructions=evaluation_agent_instructions);

evaluation_message =f"The list of emails: \\n {output}"

evaluation_result = await Runner.run(evaluation_agent,evaluation_message )

print(evaluation_result.final_output);




"Subject: Transform Your Business with Cutting-Edge Software Solutions

Hi [Recipient's Name],

I hope this email finds you well. I'm [Your Name] from BetaLaunch, where our focus is on delivering tailored software solutions to drive business success.

Are you looking to enhance your operations or expand your digital presence? We specialize in creating custom web and mobile applications designed to meet your specific needs.

Here’s how we can help:
- **Tailored Solutions:** Customized applications aligning perfectly with your business goals.
- **User-Friendly Designs:** Intuitive interfaces that enhance user engagement.
- **Scalable Platforms:** Built to grow as your business expands.

Let’s schedule a quick call to discuss how we can support your objectives. Are you available for a brief chat this week?

Looking forward to the possibility of working together.

Best regards,

[Your Full Name]  
[Your Position]  
BetaLaunch  
[Your Contact Information]"


In [60]:
@function_tool
def send_email(body: str):
    """Send an email with the given body to a reepient"""
    try:
        sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
        from_email = Email('pharshana719@gmail.com')
        to_email = To('isankadatadownload@gmail.com')
        content =  Content("text/plain", body)
        mail = Mail(from_email, to_email,"Sales Email", content).get()
        response = sg.client.mail.send.post(request_body = mail)
        return {"Status":"Success"}

        
    except Exception as e :
        print(e.message)
    

In [61]:
send_email

FunctionTool(name='send_email', description='Send an email with the given body to a reepient', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023761E3BE20>, strict_json_schema=True, is_enabled=True)

In [62]:
Description = "Write a cold email"

tool1 = sales_agent_1.as_tool(tool_name = "Write_professional_emails", tool_description= Description)
tool2 = sales_agent_2.as_tool(tool_name = "Write_user_engaging_emails", tool_description=Description)
tool3  = sales_agent_3.as_tool(tool_name = "Write_details_oriented_emails", tool_description=Description)

In [63]:
tool1

FunctionTool(name='Write_professional_emails', description='Write a cold email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Write_professional_emails_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023761E6C180>, strict_json_schema=True, is_enabled=True)

In [64]:
tools = [tool1, tool2,tool3, send_email]

In [65]:
sales_manager_instructions = """Your are are sales manager who's working for BetaLaunch. You use the tools given to generate cold sales emails\n
        You do not generate emails by yourself, instead you always use the tools\n
        You tries all the three types of email generating tools and pickup the best one by comparing the results of those three tools\n 
        The you select the email sending tool which is send_email yo send the best email. Always remember you are only sending the best selected email\n 

 """

sales_manager = Agent(name="Sales Manger", instructions=sales_manager_instructions, tools=tools, model="gpt-4o-mini")


message = "Send a cold sales email addressed to 'Dear CEO'"


with trace("Sales manager"):
    manager_resulst = await Runner.run(sales_manager, message)
    print(manager_resulst)



MaxTurnsExceeded: Max turns (10) exceeded

In [66]:
@function_tool
def send_html_email(subject: str, html_body:str) -> Dict[str, str] :
    """Send out sales emails with a give subet and formatted """
    sg = sendgrid.SendGridAPIClient(api_key= os.environ.get('SENDGRID_API_KEY'))
    from_email = Email('pharshana719@gmail.com')
    to_email = To('isankadatadownload@gmail.com')
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}

    

In [67]:

subject_agent_instructions = """Your are  capable of generating an appropriate subject for a given email message that the user is likely to response"""

html_agent_instructions = """Your are capable of converting a text email body in to a HTML email body\n 
You are given a html body which might have some markdown and you need to convert it to an HTML email body with simple, clear, compelling layout and design.
"""

subject_agent = Agent(name= "Email_Subject_Agent",instructions=subject_agent_instructions)
subject_tool = subject_agent.as_tool(tool_name="subject_writer", tool_description="Write an appropriate subejct for the sales email")

html_agent = Agent(name="HTML_converter", instructions=html_agent_instructions)
html_tool = html_agent.as_tool(tool_name="HTML_converter_tool",tool_description="Convert the text email body in to HTML formatted email body")


In [68]:
tools = [subject_tool, html_tool, send_html_email]

In [69]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

email_agent = Agent(name="Email_sender",
                     instructions=instructions,
                     model="gpt-4o-mini",
                     tools=tools,
                     handoff_description="Convert an email to HTML and send it")

In [74]:
sales_manager_instructions = """You are a sales manager working for BetaLaunch. You use tools given to generate sales emails\n 
You never generate emails by yourself, instead you use tools. You try all the three email generating tools and pick up the best response among them\n 
You can use a tool more than once if a tool did not provide and appropriate email body. Then you select the best email body among them that a user will
most likeyly to reply to.\n After selecting the best email body you handoff to the Email Manager Agent to format and send the email. 

"""

new_tools_list = [tool1, tool2, tool3]
handoffs = [email_agent]

new_sales_manager = Agent(
    name="New_sales_manager",
    instructions=sales_manager_instructions,
    tools=new_tools_list,
    handoffs=handoffs,
    model="gpt-4o-mini"
)
message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(new_sales_manager, message);